In [2]:
from scipy.spatial import distance
import numpy as np 
import pandas as pd 
import os
import cv2
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
import torch
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torchvision import transforms as T
from torchvision import transforms
import pytorch_msssim
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import torch.nn as nn
from tqdm import tqdm
from torch.optim import lr_scheduler
import gc
from time import sleep
import math


train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



2024-11-22 00:20:36.269250: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-22 00:20:36.282665: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-22 00:20:36.286480: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-22 00:20:36.296649: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-22 00:20:37.032546: W tensorflow/compiler/tf2

CUDA is available!  Training on GPU ...


In [3]:


class Drone:
    def __init__(self, id, x, y, z):
        self.id = id
        self.x = x
        self.y = y
        self.z = z

class Encoder(nn.Module):

    def __init__(self, encoded_space_dim):
        super().__init__()
        
        self.encoded_space_dim = encoded_space_dim
        ### Convolutional section
        self.encoder_cnn = nn.Sequential(
            nn.Conv2d(3, 8, 4, stride=2, padding=0),
            nn.ReLU(True),
            nn.Conv2d(8, 16, 4, stride=2, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(True),
            nn.Conv2d(16, 32, 4, stride=2, padding=0),
            nn.ReLU(True),
            nn.Conv2d(32, 64, 4, stride=2, padding=0),
            nn.ReLU(True),
            nn.Conv2d(64, 128, 4, stride=2, padding=1),
            nn.ReLU(True)
        )
        
        ### Flatten layer
        self.flatten = nn.Flatten(start_dim=1)
### Linear section
        self.encoder_lin = nn.Sequential(
            nn.Linear(8 * 8 * 128, 1024),
            nn.ReLU(True),
            nn.Linear(1024, encoded_space_dim)
        )
        
    def forward(self, x):
        x = self.encoder_cnn(x)
        x = self.flatten(x)
        x = self.encoder_lin(x)
        #z_mean = self.encoder_lin(x)
        #z_log_var = self.encoder_lin(x)
        #N = torch.normal(0, 1, size=(z_log_var.size()[0], self.encoded_space_dim))
        #N = N.to(device)
        #print('N: ', N.shape, ' z_log_var: ', z_log_var.shape)
        return x #(torch.exp(z_log_var / 2) * N + z_mean), z_mean, z_log_var

def load_embeddings():
    image_path = '../map/test_map_crop.png'
    full_map = cv2.imread(image_path)
    full_map = cv2.cvtColor(full_map, cv2.COLOR_BGR2RGB)
    # координаты тайлов
    height, width, _ = full_map.shape
    step = 50
    tile_size = 300
    coord_dataset = pd.DataFrame(columns = ['x', 'y'])
    for x in range(0, width - tile_size, step):
        for y in range(0, height - tile_size, step):
            coord_dataset.loc[len(coord_dataset.index)] = [x, y]
    class CustomDataset(Dataset):
        def __init__(self, full_map, coord_dataset, tile_size, transform=None):
            self.mapa = full_map
            self.coords = coord_dataset
            self.tile_size = tile_size
            self.transform = transform

        def __len__(self):
            return len(self.coords.index)

        def __getitem__(self, idx):
            x = self.coords.iloc[idx]['x']
            y = self.coords.iloc[idx]['y']
            image = full_map[y:y+tile_size, x:x+tile_size]
            if self.transform:
                image = self.transform(image)
                
            return image, (x, y)
    encoder = Encoder(encoded_space_dim=256)

    best_encoder = torch.load('./models/best_encoder_02_08.pth', weights_only=True)
    encoder.load_state_dict(best_encoder)   
      

    batch_size = 1
    train_dataset = CustomDataset(full_map=full_map[:, :5000], coord_dataset = coord_dataset, tile_size = tile_size, transform=transforms.ToTensor())
    dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    encoder.to(device)
    encoder.eval()
    embedings = []
    labels = []
    with torch.no_grad():
        for data in dataloader:
            # if i == 1:
            #     break
            inputs, label = data
            # print("Input: ", inputs)
            # print("Labels: ", labels)
            # print("Input: ", inputs.shape)
            inputs = inputs.to(device)
            coded= encoder(inputs)
    
            inputs = inputs.cpu().numpy().flatten().tolist()
            coded = coded.cpu().numpy().flatten().tolist()

            
            # print(coded)
            # print(coded.shape)
            embedings.append(coded)
            labels.append(label)
            # print("---------------")
            

    print("Emb Amount: ", len(embedings))
    print("Emb: ", embedings[0:3], "...")
    return labels, embedings

def image_to_tensor(drone):
    # пока использую статичное изображение, потом буду получать от камеры
    # step = 50
    tile_size = 300
    x = drone.x
    y = drone.y
    full_map_image_path = '../map/test_map_crop.png'
    full_map = cv2.imread(full_map_image_path)
    fpv_image = full_map[y:y+tile_size, x:x+tile_size]
    fpv_image = cv2.cvtColor(fpv_image, cv2.COLOR_BGR2RGB)
    transform=transforms.ToTensor()
    fpv_image = transform(fpv_image)
    fpv_image = fpv_image.unsqueeze(0)
    print("Image shape:",fpv_image.shape)
    return fpv_image

def image_to_embeding(inputs):
     
    encoder = Encoder(encoded_space_dim=256)
    encoder.to(device)
    best_encoder = torch.load('./models/best_encoder_02_08.pth')
    encoder.load_state_dict(best_encoder)  
    encoder.eval()
    with torch.no_grad():
        inputs = inputs.to(device)
        coded= encoder(inputs)
        coded = coded.cpu().numpy()
        print("Image coded emb shape:",coded.shape)
        return coded



In [6]:
# загржуам эмбединги в память
labels, data = load_embeddings()




Emb Amount:  11766
Emb:  [[-109063.4296875, -1610054.875, 1008138.0625, -736228.6875, -1418765.875, 1002615.625, -2081069.125, -745765.5, -1122884.625, 1502688.25, -886512.5, 685118.625, 52706.7734375, -1387809.125, 941735.0, -1431123.25, -286403.0625, 1070841.5, 1524850.5, -1458484.625, -372061.21875, 2081430.5, -443928.21875, -85036.640625, -971231.625, -483729.25, 2338120.5, -649905.4375, 712542.5625, -1242099.5, -1110719.625, 1282539.25, 1860087.75, 238650.296875, -1318332.0, -374821.28125, -688004.5, 712909.9375, -1764001.75, 1212028.75, -1526510.0, -1492548.75, -1174035.0, 88268.1171875, 1726629.875, 749449.25, 129111.3828125, -1067630.75, 60677.328125, -814763.375, 1049541.375, 222225.8125, -1435040.625, 444562.9375, -1051012.375, -1738129.0, 810120.0625, 684892.8125, -2165921.5, -234708.734375, -1744475.625, 1143987.375, -897471.25, 752803.6875, 296661.34375, 180716.203125, 1820552.625, 711689.3125, 899685.1875, -1417080.625, -1451214.125, -1540277.0, -667586.375, 353138.21875,

In [5]:
# ключ-значение для меток
embedings = dict(zip(map(tuple,data), labels))

# Преобразуем embeddings в DataFrame
df = pd.DataFrame(data)
print(df.shape)


(11766, 256)


In [8]:
import pickle

# Функция для сохранения словаря с эмбеддингами
def save_embeddings(embeddings, filename):
    try:
        with open(filename, 'wb') as file:
            pickle.dump(embeddings, file)
        print(f"Эмбеддинги успешно сохранены в файл {filename}")
    except Exception as e:
        print(f"Ошибка при сохранении эмбеддингов: {e}")

# Функция для загрузки словаря с эмбеддингами
def load_embeddings(filename):
    try:
        with open(filename, 'rb') as file:
            embeddings = pickle.load(file)
        print(f"Эмбеддинги успешно загружены из файла {filename}")
        return embeddings
    except Exception as e:
        print(f"Ошибка при загрузке эмбеддингов: {e}")
        return None

# Пример использования:
# Сохранение
embeddings = dict(zip(map(tuple, data), labels))
save_embeddings(embeddings, "embeddings.pkl")

# Загрузка
# loaded_embeddings = load_embeddings("embeddings.pkl")

Эмбеддинги успешно сохранены в файл embeddings.pkl


/home/kravtandr/.local/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


Эмбеддинги успешно загружены из файла embeddings.pkl


In [9]:

# Вычисляем ковариационную матрицу для embedings и инвертируем её
cov_matrix = np.cov(df.T)
inv_cov_matrix = np.linalg.inv(cov_matrix)

# drone start coords
drone = Drone(0, 333, 777, 0)

    

In [10]:
import numpy as np

def save_matrix(matrix, filename):
    """
    Сохраняет матрицу в файл
    
    Параметры:
    matrix: numpy array - матрица для сохранения
    filename: str - имя файла для сохранения (с расширением .npy)
    """
    try:
        np.save(filename, matrix)
        print(f"Матрица успешно сохранена в файл {filename}")
    except Exception as e:
        print(f"Ошибка при сохранении матрицы: {e}")

def load_matrix(filename):
    """
    Загружает матрицу из файла
    
    Параметры:
    filename: str - имя файла для загрузки (с расширением .npy)
    
    Возвращает:
    numpy array - загруженная матрица
    """
    try:
        matrix = np.load(filename)
        print(f"Матрица успешно загружена из файла {filename}")
        return matrix
    except Exception as e:
        print(f"Ошибка при загрузке матрицы: {e}")
        return None

# Пример использования:
# Предположим, у вас есть матрица inv_cov_matrix

# Сохранение матрицы
save_matrix(inv_cov_matrix, 'inv_cov_matrix.npy')

# Загрузка матрицы
# loaded_matrix = load_matrix('inv_cov_matrix.npy')

Матрица успешно сохранена в файл inv_cov_matrix.npy


In [135]:
def nearbyAreaCheck(drone, embedings, embeding_vector):
    nearbyDelta = 300
    x = embedings[tuple(embeding_vector)][0]
    y = embedings[tuple(embeding_vector)][1]
    if abs(x-drone.x) < nearbyDelta and abs(y-drone.y) < nearbyDelta:
        return True
    else:
        return False

In [136]:
# while True:
for i in range(1):
    # get image from camera
    fpv_image = image_to_tensor(drone)

    # image to encoder
    encoded_image = image_to_embeding(fpv_image).flatten()

    min_mahalanobis_distance = np.inf
    closest_embeding_vector = None
    # тут нужно потом сравнивать не со всеми а только с ближайшими
    for embeding_vector in data:
        if nearbyAreaCheck(drone, embedings, embeding_vector):
            mahalanobis_distance = distance.mahalanobis(encoded_image, embeding_vector, inv_cov_matrix)
            # print(mahalanobis_distance)
            if mahalanobis_distance < min_mahalanobis_distance:
                min_mahalanobis_distance = mahalanobis_distance
                closest_embeding_vector = embeding_vector

    print("Min mahal: ",min_mahalanobis_distance)
    print("New Drone coords: ",embedings[tuple(closest_embeding_vector)])
    print("Real Drone coords: ", drone.x, drone.y)

Image shape: torch.Size([1, 3, 300, 300])
Image coded emb shape: (1, 256)


/tmp/ipykernel_4718/1534400846.py:176: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_encoder = torch.load('/media/kravtandr/SSD_1TB/Drone_nav/maks/best_encoder_02_08.pt

Min mahal:  9.003372880059008
New Drone coords:  [tensor([350]), tensor([750])]
Real Drone coords:  333 777
